In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [5]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-1]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [6]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_2.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.]
Size: 16


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [7]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [8]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:248]
    features_test_csv  = features_csv_files[248:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_data_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_data_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 298
Train matches: 248
Test matches: 50
Train-set shape: (569420, 228)
Test-set shape: (109962, 228)


In [9]:
    # shuffling data-sets agin
    # df_train = df_train.drop('homeGoalKeeperY', axis=1)
    
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [10]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (569420, 228)

Pitch index counts
------------------
0	: 176277
1	: 11098
2	: 27723
3	: 10323
4	: 27644
5	: 27811
6	: 26573
7	: 38334
8	: 39415
9	: 38770
10	: 28787
11	: 28243
12	: 30886
13	: 12058
14	: 31356
15	: 14122


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,54.093750,38.65625,0.939941,1.0,-1.0,60.53125,28.59375,51.84375,31.171875,72.3125,...,41.781250,2.130859,0.00000,0.000,38.3750,41.78125,0.000,0.000000,0.00000,8.0
1,65.750000,24.15625,3.289062,-1.0,-1.0,71.06250,33.06250,74.56250,38.875000,81.1875,...,24.015625,4.468750,0.00000,0.000,0.0000,0.00000,51.625,24.015625,4.46875,7.0
2,27.890625,35.37500,1.330078,1.0,-1.0,26.09375,32.90625,26.87500,33.718750,47.1250,...,39.031250,1.509766,0.00000,0.000,13.2500,39.03125,0.000,0.000000,0.00000,0.0
3,65.625000,35.78125,3.990234,-1.0,1.0,56.34375,37.31250,54.40625,37.125000,72.5000,...,38.437500,2.830078,0.00000,0.000,45.3125,38.43750,0.000,0.000000,0.00000,12.0
4,76.312500,37.34375,2.539062,1.0,-1.0,86.00000,43.37500,68.18750,46.437500,91.0000,...,49.125000,1.349609,50.34375,49.125,0.0000,0.00000,0.000,0.000000,0.00000,0.0


### Test Set

In [11]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (109962, 228)

Pitch index counts
------------------
0	: 33460
1	: 2150
2	: 5279
3	: 2138
4	: 5132
5	: 5251
6	: 5472
7	: 7692
8	: 7842
9	: 7989
10	: 5367
11	: 5321
12	: 6002
13	: 2374
14	: 5887
15	: 2606


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,59.375000,31.81250,1.339844,-1.0,1.0,60.31250,34.53125,52.87500,42.625000,69.8750,...,47.90625,2.580078,0.0000,0.0000,48.09375,47.90625,0.000000,0.00,0.000000,6.0
1,90.687500,34.56250,2.630859,-1.0,-1.0,58.87500,32.96875,50.68750,27.078125,73.9375,...,35.93750,0.226685,47.5625,35.9375,0.00000,0.00000,0.000000,0.00,0.000000,0.0
2,63.125000,33.68750,2.210938,1.0,-1.0,58.81250,39.28125,58.18750,36.187500,72.8750,...,28.53125,1.764648,0.0000,0.0000,50.65625,28.53125,0.000000,0.00,0.000000,8.0
3,51.718750,37.18750,2.859375,1.0,-1.0,53.21875,48.84375,51.81250,42.937500,59.2500,...,21.50000,4.449219,0.0000,0.0000,0.00000,0.00000,38.000000,21.50,4.449219,2.0
4,25.578125,33.21875,4.550781,-1.0,1.0,39.62500,29.46875,35.53125,28.531250,65.5625,...,35.75000,3.460938,0.0000,0.0000,0.00000,0.00000,18.734375,35.75,3.460938,2.0


## Exporting

In [12]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}_2.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}_2.csv'.format(dataset='test'), index=False)

In [13]:
    df_train[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,65.750000,24.156250,3.289062,-1.0,-1.0,71.06250,33.062500,74.562500,38.875000,81.18750,...,24.015625,4.468750,0.00000,0.000000,0.00000,0.000000,51.625,24.015625,4.46875,7.0
2,27.890625,35.375000,1.330078,1.0,-1.0,26.09375,32.906250,26.875000,33.718750,47.12500,...,39.031250,1.509766,0.00000,0.000000,13.25000,39.031250,0.000,0.000000,0.00000,0.0
3,65.625000,35.781250,3.990234,-1.0,1.0,56.34375,37.312500,54.406250,37.125000,72.50000,...,38.437500,2.830078,0.00000,0.000000,45.31250,38.437500,0.000,0.000000,0.00000,12.0
4,76.312500,37.343750,2.539062,1.0,-1.0,86.00000,43.375000,68.187500,46.437500,91.00000,...,49.125000,1.349609,50.34375,49.125000,0.00000,0.000000,0.000,0.000000,0.00000,0.0
5,57.937500,27.140625,1.169922,1.0,-1.0,52.12500,34.156250,44.656250,34.750000,68.50000,...,48.125000,1.405273,43.59375,48.125000,0.00000,0.000000,0.000,0.000000,0.00000,5.0
6,51.218750,41.468750,0.680176,1.0,-1.0,48.71875,26.828125,53.500000,30.921875,75.75000,...,31.437500,1.190430,40.15625,31.437500,0.00000,0.000000,0.000,0.000000,0.00000,11.0
7,63.718750,33.062500,1.469727,-1.0,-1.0,57.75000,33.187500,52.125000,36.468750,69.06250,...,27.453125,2.140625,0.00000,0.000000,42.68750,27.453125,0.000,0.000000,0.00000,5.0
8,33.500000,14.281250,0.770020,-1.0,-1.0,32.03125,21.390625,28.265625,22.000000,41.65625,...,27.421875,1.254883,35.18750,27.421875,0.00000,0.000000,0.000,0.000000,0.00000,0.0
9,59.593750,37.125000,2.060547,1.0,-1.0,59.21875,46.375000,50.968750,40.156250,73.06250,...,48.218750,2.474609,0.00000,0.000000,45.46875,48.218750,0.000,0.000000,0.00000,0.0
